# 在只有Faiss语义检索器中加入语义相似度


In [ ]:
!pip install pypdfium2 backoff openai langchain_openai langchain langchain_community faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.2/437.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstal

配置OpenAI 的 GPT-4o 模型

In [ ]:
# 设置OpenAI KEY环境变量
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [ ]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_documents(
    docs, embedding=OpenAIEmbeddings()
)

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain


@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*faiss_vectorstore.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [ ]:
result = retriever.invoke("dinosaur")
result

(Document(id='99484148-5c13-40ab-9c8c-273d89d722d4', metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction', 'score': 0.3112905}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(id='8767a9dd-87ff-4eca-85e3-ea58caccfc43', metadata={'year': 1995, 'genre': 'animated', 'score': 0.41597342}, page_content='Toys come alive and have a blast doing so'),
 Document(id='67bd781d-d3d4-4e7e-9d1e-8a7e8d2d588e', metadata={'year': 1979, 'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'score': 0.4970406}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(id='7ea07c65-1eab-49b5-bdfc-735768d42996', metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6, 'score': 0.50489014}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'))

# 通过检索器来实现

In [ ]:
# 查询向量存储
query = "dinosaur"
faiss_retriever = faiss_vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.0}
)
docs = faiss_retriever.get_relevant_documents(query)
# 打印结果
for doc in docs:
    print(doc.page_content)

<ipython-input-6-8f57519dd730>:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = faiss_retriever.get_relevant_documents(query)


A bunch of scientists bring back dinosaurs and mayhem breaks loose
Toys come alive and have a blast doing so
Three men walk into the Zone, three men walk out of the Zone
A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea


上述证明两种方法检索到的内容是一样的

In [ ]:
docs

[Document(id='99484148-5c13-40ab-9c8c-273d89d722d4', metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction', 'score': 0.3112905}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(id='8767a9dd-87ff-4eca-85e3-ea58caccfc43', metadata={'year': 1995, 'genre': 'animated', 'score': 0.41597342}, page_content='Toys come alive and have a blast doing so'),
 Document(id='67bd781d-d3d4-4e7e-9d1e-8a7e8d2d588e', metadata={'year': 1979, 'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'score': 0.4970406}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(id='7ea07c65-1eab-49b5-bdfc-735768d42996', metadata={'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6, 'score': 0.50489014}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea')]

In [ ]:
# 打印结果
for doc in docs:
    print(doc.metadata)

{'year': 1993, 'rating': 7.7, 'genre': 'science fiction', 'score': 0.3112905}
{'year': 1995, 'genre': 'animated', 'score': 0.41597342}
{'year': 1979, 'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'score': 0.4970406}
{'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6, 'score': 0.50489014}


In [ ]:
# 打印结果
for doc in docs:
    print(doc.metadata["score"])

0.3112905
0.41597342
0.4970406
0.50489014
